In [2]:
import sys
from pathlib import Path

# Add the project root directory to Python path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

In [3]:
from utils.model_interactions import build_prompt
from config import MODELS_PATH, MODEL_ID

In [4]:
import os

os.environ["PYTHONIOENCODING"] = "utf-8"

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.pipelines import pipeline
from peft import PeftModel, PeftConfig
import torch

model_path = MODELS_PATH / "qwen-lora-finetuned"

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_ID, 
    trust_remote_code=True
)
base = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype = torch.float16,
    device_map = "cpu",
    trust_remote_code = True
)

model = PeftModel.from_pretrained(
    base,
    model_path,
    torch_dtype = torch.float16
)
model = model.merge_and_unload()
# Move to device after merging
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu


In [10]:
example = {
    "system": "Ты — преподаватель немецкого языка для русскоязычных студентов. Отвечай чётко, понятно и методически правильно.",
    "instruction": "Проанализируй диалог и укажи грамматические ошибки.\n\nKunde: Ich möchte ein Milch.\nVerkäufer: Eine Milch oder einen Saft?\nKunde: Eine Milch, bitte.",
}

prompt = build_prompt(example["system"], example["instruction"])

output = list(pipe(prompt, max_new_tokens=300, do_sample=True, temperature=0.7))[0]["generated_text"]

print(output[len(prompt):].strip()) 


В этом диалоге нет грамматических ошибок, но есть несколько моментов, которые можно улучшить с точки зрения языковой правильности и культурного контекста.

1. Kunde: Ich möchte einen Milch. 
Клиент хочет сказать "я хочу молока", но в немецком языке после глагола "möchte" (хочу) используется артикль "einen". В русском языке это выражение звучит как "Я хочу молока", поэтому в немецком языке клиент должен сказать "Ich möchte einen Milch." или "Ich möchte eine Milch." в зависимости от контекста. В данном случае клиент хочет молоко, поэтому он должен использовать "einen". 

2. Verkäufer: Eine Milch oder einen Saft? 
Продавец предлагает два варианта: молоко или сок. В немецком языке это выражение звучит как "Eine Milch oder einen Saft?" в данном случае нет ошибок, но клиент уже выбирал молоко, поэтому продавец может сказать "Eine Milch, bitte." 

3. Kunde: Eine Milch, bitte. 
Клиент уже выбрал молоко, поэтому продавец может закончить диалог, не спрашивая дополн
